In [14]:
from transformers import AutoTokenizer, BertModel, BertConfig
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = BertModel.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [15]:
side_effects = []
side_effects_path = './data/side_effects.txt'
with open(side_effects_path) as f:
    for line in f:
        side_effects.append(line.strip())

In [22]:
batch_size = 64

for i in range(0, len(side_effects), batch_size):
    batch = side_effects[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states)
    # torch.save(last_hidden_states, f'../data/side_effects_bert_embeddings_{i}.pt')
    print(f'Batch {i} done')
    break

print(last_hidden_states.shape)

tensor([[[-9.2741e-01, -3.3870e-01, -3.6889e-01,  ...,  8.1184e-03,
          -1.1107e-01,  4.5286e-01],
         [-1.1543e+00,  3.0090e-01, -1.8213e-01,  ...,  2.9203e-01,
          -1.1849e-01, -2.9007e-01],
         [-6.3014e-01, -7.9829e-01, -7.9640e-01,  ...,  6.4584e-01,
           2.3017e-02,  1.4699e-01],
         ...,
         [-6.7741e-01, -3.9072e-01, -3.0077e-01,  ...,  3.0403e-01,
          -1.3635e-01,  6.9805e-01],
         [-5.5073e-01, -1.3315e-01, -1.9100e-01,  ...,  2.1301e-01,
          -1.5731e-01,  6.4711e-01],
         [-6.1202e-01, -1.3520e-01, -1.9754e-01,  ...,  2.2133e-01,
          -1.9033e-01,  6.3468e-01]],

        [[-5.2713e-01,  3.5818e-02, -6.3200e-01,  ...,  3.8279e-02,
          -1.1523e-02,  5.8164e-01],
         [-5.3111e-01,  5.6707e-01, -3.7145e-01,  ...,  8.6685e-02,
           8.3947e-02,  3.3356e-01],
         [-5.1219e-01,  7.1514e-01, -2.1655e-01,  ...,  8.2671e-02,
          -6.8099e-01,  2.5449e-01],
         ...,
         [-4.1345e-01, -2

In [21]:
flattened_vecs = last_hidden_states.view(64, -1).detach()
print(flattened_vecs.shape)

torch.Size([64, 8448])


In [19]:
from utils.similarity import *

similarity_matrix = cosine_similarity(flattened_vecs)
plot_similarity_matrix(similarity_matrix, side_effects)

ValueError: Image size of 20434804x455 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 640x480 with 2 Axes>